In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
import pandas as pd

# 데이터 불러오기

### 각 카테고리별로 크롤링한 데이터 병합

In [ ]:
dy = [220, 310, 320, 400, 405, 410, 420, 430, 500]
ty = [600, 700, 750, 800, 810, 900]
mh = [910, 920, 930, 980, 990, 999]
df = pd.DataFrame()
for i in dy+ty+mh:
    temp = pd.read_csv(f"data/crawling/{i}.csv")
    if 'warning' in temp.columns:
        temp = temp.drop('warning', axis=1)
    if 'address_id' in temp.columns:
        temp = temp.drop('address_id', axis=1)
    df = df.append(temp)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


데이터 개수 확인

In [ ]:
len(df)

604716

컬럼 종류 확인

In [ ]:
df.columns

Index(['pid', 'uid', 'name', 'status', 'price', 'num_faved', 'num_item_view',
       'num_comment', 'user_name', 'update_time', 'free_shipping',
       'profile_image', 'product_image', 'bizseller', 'category_id',
       'category_name', 'tradable', 'used', 'used_code', 'image_count',
       'image_file_list_for_edit', 'image_source', 'location', 'latitude',
       'longitude', 'qty', 'keyword', 'contact_hope', 'bunpay',
       'need_induce_bun_pay_filter', 'is_location_confirm',
       'only_neighborhood', 'neighborhood_option', 'pay_option',
       'restriction_status', 'is_ad', 'is_adult', 'style', 'group_ids',
       'groups', 'is_buncar', 'checkout', 'naver_shopping_url',
       'extended_spec', 'ordernow_label', 'ordernow_token_required',
       'contact_enabled', 'comment_enabled', 'is_free_sharing', 'num_item',
       'num_grade_avg', 'num_follower', 'num_review', 'is_withdraw',
       'is_blocked', 'join_date', 'access_time', 'is_identification',
       'badges'],
      dtype=

## 전처리

### 기존 데이터 형식에 맞게 이름 변경

In [ ]:
rename_dict = {'pid': 'content_id', 
               'uid': 'adv_id', 
               'name': 'content_name',
               'status': 'content_status',
               'price': 'content_price',
               'num_faved': 'content_likes',
               'num_item_view': 'content_views',
               'num_comment': 'content_comment_count', 
               'user_name': 'adv_name',
               'free_shipping': 'content_delivery_fee',
               'used_code': 'content_used',
               'location': 'content_place',
               'need_induce_bun_pay_filter': 'content_b_pay',
               'num_item': 'adv_item_count',
               'num_grade_avg': 'adv_grade',
               'num_follower': 'adv_follower_count',
               'num_review': 'adv_review_count',
           }


### drop할 컬럼 목록 정리

In [ ]:
drop_list = ['contact_enabled', 'naver_shopping_url', 'bizseller', 'groups', 'is_withdraw',
            'category_id', 'ordernow_token_required', 'only_neighborhood', 'is_identification',
            'is_adult', 'style', 'group_ids', 'ordernow_label', 'checkout', 'is_free_sharing',
            'is_blocked', 'extended_spec', 'contact_hope', 'used', 'comment_enabled',
            'restriction_status', 'is_buncar', 'badges', 'category_name', 'product_image', 'pay_option']

### 장소 데이터를 시/도 형식으로 변환

- 결측치는 `전국`으로 처리
- 아래 딕셔너리로 변환했을 때 변환이 안되는 값들이 전체 60만개 중 1000개정도 존재  
    -> `전국`으로 처리

In [ ]:
place_dict = {
    '서울': '서울특별시',
    '강남': '서울특별시',
    '잠실': '서울특별시',
    '경기도': '경기도', '경기': '경기도',
    '강원': '강원도', '제주': '제주도',
    '경상북': '경상북도', '경북': '경상북도',
    '경상남': '경상남도', '경남': '경상남도',
    '전라북': '전라북도', '전북': '전라북도',
    '전라남': '전라남도', '전남': '전라남도',
    '충청북': '충청북도', '충북': '충청북도',
    '충청남': '충청남도', '충남': '충청남도',
    '인천': '인천광역시',
    '대구': '대구광역시',
    '대전': '대전광역시',
    '부산': '부산광역시',
    '울산': '울산광역시',
    '광주': '광주광역시',
    '세종': '세종시'
}

def sido_new(place):
    if isinstance(place, str):
        for name in place_dict.keys():
            if name in place:
                place = place_dict[name]
    if place not in place_dict.values():
        place = '전국'
    return place

### 컬럼 순서 보기 좋게 정렬

In [ ]:
sort_col = ['content_id', 'content_name', 'adv_id', 'adv_name', 'keyword', 'content_status',
       'content_price', 'content_used', 'content_likes', 'content_views',
       'content_comment_count', 'content_delivery_fee',
       'content_cat_1', 'content_cat_2', 'content_cat_3',
       'content_b_pay', 
       'adv_item_count', 'adv_grade', 'adv_follower_count', 'adv_review_count',
       'tradable', 'qty', 
       'content_place', 'latitude', 'longitude', 'is_location_confirm', 
       'image_count', 'image_file_list_for_edit', 'image_source', 'profile_image',
       'neighborhood_option', 'is_ad', 'bunpay', 'pay_option_in_person', 'pay_option_bun_pay_filter_enabled', 
       'update_time', 'access_time', 'join_date', 
       'seller_name']

### 위 과정을 모아서 전처리

In [ ]:
def preprocess_crawling(df_):
    df = df_.copy()
    df = df.rename(rename_dict, axis=1)
    # 중복 제거
    df = df.drop_duplicates('content_id')
    df = df.reset_index().drop('index', axis=1)

    # 대분류
    df['content_cat_1'] = df['category_name'].apply(lambda x : x.split("'")[3])
    # 중분류
    df['content_cat_2'] = df['category_name'].apply(lambda x : x.split("'")[7] if len(x.split("'"))>6 else x.split("'")[3])
    # 소분류
    df['content_cat_3'] = df['category_name'].apply(lambda x : x.split("'")[-2])
    # delivery_fee -> train 데이터와 같게
    df['content_delivery_fee'] = df['content_delivery_fee'].apply(lambda x : 1*x)
    # b_pay -> train 데이터와 같게
    df['content_b_pay'] = df['content_b_pay'].apply(lambda x : 1 - 1*x)
    # 중고 여부를 train 데이터와 같게
    df['content_used'] = df['content_used'].apply(lambda x: 0 if x==2 else 1)
    # 지역명을 시/도 형식에 맞게
    df['content_place'] = df['content_place'].apply(sido_new)
    # pay option에서 필요한 부분만 사용
    df['pay_option_in_person'] = df['pay_option'].apply(lambda x : x.split("'")[4][2:-2])
    df['pay_option_bun_pay_filter_enabled'] = df['pay_option'].apply(lambda x : x.split("'")[6][2:-2])
    # unix time을 실제 시간으로
    df['update_time'] = pd.to_datetime(df['update_time'], unit='s')
    df['access_time'] = pd.to_datetime(df['access_time'], unit='s')
    df['join_date'] = pd.to_datetime(df['join_date'], unit='s')
    
    df['seller_name'] = df['badges'].apply(lambda x : x[94:97])

    # 필요없는 컬럼 drop
    df = df.drop(drop_list, axis=1)
    # 보기 좋게 정렬
    df = df[sort_col]
    return df

위에서 병합한 데이터 전처리 진행

In [ ]:
df = preprocess_crawling(df)

In [ ]:
df = df.drop(['image_file_list_for_edit', 'profile_image'], axis=1)

In [ ]:
df

,content_id,content_name,adv_id,adv_name,keyword,content_status,content_price,content_used,content_likes,content_views,content_comment_count,content_delivery_fee,content_cat_1,content_cat_2,content_cat_3,content_b_pay,adv_item_count,adv_grade,adv_follower_count,adv_review_count,tradable,qty,content_place,latitude,longitude,is_location_confirm,image_count,image_source,neighborhood_option,is_ad,bunpay,pay_option_in_person,pay_option_bun_pay_filter_enabled,update_time,access_time,join_date,seller_name
0,151575858,개인용달/운송/상담/원룸이사,3826804,스프라이트7,NaN,0,0,1,1,63,0,0,지역 서비스,이사/용달,이사/용달,1,2904,10,4,3,False,1,서울특별시,37.599007,127.035160,1,1,ALBUM,display_on_neighborhood,False,False,True,False,2022-01-03 04:46:31,2021-12-26 16:01:00,2015-08-17 07:23:19,*시*
1,151575805,개인용달/운송/상담/원룸이사,3826804,스프라이트7,NaN,0,0,1,0,51,0,0,지역 서비스,이사/용달,이사/용달,1,2904,10,4,3,False,1,서울특별시,37.599007,127.035160,1,1,ALBUM,display_on_neighborhood,False,False,True,False,2022-01-03 04:46:31,2021-12-26 16:01:00,2015-08-17 07:23:19,*시*
2,151575764,개인용달/운송/상담/원룸이사,3826804,스프라이트7,NaN,0,0,1,0,44,0,0,지역 서비스,이사/용달,이사/용달,1,2904,10,4,3,False,1,서울특별시,37.599007,127.035160,1,1,ALBUM,display_on_neighborhood,False,False,True,False,2022-01-03 04:46:31,2021-12-26 16:01:00,2015-08-17 07:23:19,*시*
3,151096237,개인용달/운송/상담/원룸이사,3826804,스프라이트7,NaN,0,0,1,0,50,0,0,지역 서비스,이사/용달,이사/용달,1,2904,10,4,3,False,1,서울특별시,37.599007,127.035160,1,1,ALBUM,display_on_neighborhood,False,False,True,False,2022-01-03 04:46:31,2021-12-26 16:01:00,2015-08-17 07:23:19,*시*
4,151096173,개인용달/운송/상담/원룸이사,3826804,스프라이트7,NaN,0,0,1,0,42,0,0,지역 서비스,이사/용달,이사/용달,1,2904,10,4,3,False,1,서울특별시,37.599007,127.035160,1,1,ALBUM,display_on_neighborhood,False,False,True,False,2022-01-03 04:46:31,2021-12-26 16:01:00,2015-08-17 07:23:19,*시*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575955,165335407,빈티지 향수병 퍼퓸 보틀 스프레이,2019468,핑뀨핑뀨,"빈티지퍼퓸보틀,빈티지향수병,빈티지퍼퓸",0,53000,1,8,110,0,1,기타,기타,기타,1,62,10,34,19,False,1,경상북도,36.156865,128.585400,0,2,ALBUM,not_verified,False,True,True,True,2021-12-27 15:36:09,2021-12-24 02:31:23,2014-02-10 07:21:50,*다*
575956,164037328,빈티지 플라워액자 거울,2019468,핑뀨핑뀨,빈티지액자,0,62000,1,17,276,0,1,기타,기타,기타,1,62,10,34,19,False,1,경상북도,36.156865,128.585400,0,3,ALBUM,not_verified,False,True,True,True,2021-12-27 15:36:09,2021-12-24 02:31:23,2014-02-10 07:21:50,*다*
575957,163303957,빈티지 빅토리안 앤틱박스 보석함,2019468,핑뀨핑뀨,"빈티지보석함,앤틱박스,앤틱보석함,빈티지박스",0,185000,1,2,194,0,1,기타,기타,기타,1,62,10,34,19,False,1,경상북도,36.156865,128.585400,0,6,ALBUM,not_verified,False,True,True,True,2021-12-27 15:36:09,2021-12-24 02:31:23,2014-02-10 07:21:50,*다*
575958,172052479,폴로 패딩 xl,75531916,nat,폴로패딩,0,88000,1,13,307,0,0,기타,기타,기타,1,87,10,9,2,False,1,서울특별시,37.474989,126.934991,0,2,NaN,not_verified,False,True,True,True,2021-12-27 15:35:41,2021-12-06 16:30:19,2021-02-22 07:03:56,*지*


In [ ]:
df.to_csv('data/content.csv', index=False)

# CTR 예측 모델 학습(기존 데이터 사용)

## 학습 데이터 준비

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
train_data = pd.read_csv('data/train.csv')

### 범주형/연속형 feature 구분

In [ ]:
categorical = [
    'viewer_gender',  
    'content_used',
    'content_cat_1',
    'content_delivery_fee',
    'content_b_pay'
    ]
    
continuous = [
    'bid_price', 
    'content_price',
    'content_comment_count',     
    'content_views',       
    'content_likes',
    'adv_item_count',
    'adv_follower_count', 
    'adv_review_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    "viewer_parcel_post_count",
    'viewer_transfer_count',
    'viewer_chat_count', 
    ]

features = categorical + continuous

### 전처리

In [ ]:
# 수동 라벨인코딩
cat_label = {}
for i, cat in enumerate(train_data['content_cat_1'].unique()):
    cat_label[cat] = i

train_data['content_cat_1'] = train_data['content_cat_1'].apply(lambda x: cat_label[x])

train_data['content_delivery_fee'] = train_data['content_delivery_fee'].apply(lambda x: 1 if x=='배송비포함' else 0)

# train_data['viewer_age'] = train_data['viewer_age'].apply(lambda x: x//10 * 10)
# 스케일링, scaler 남겨두기
scaler = StandardScaler()
# scaler = RobustScaler()
train_data[continuous] = scaler.fit_transform(train_data[continuous])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data['label'], test_size=0.2, random_state=47)

### 평가지표 정의

In [ ]:
average_ctr = train_data['label'].mean()

# 평가지표 함수

def get_rig(train_y, test_y, pred, avg_ctr):
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig

def evaluate(pred):
    print("")
    print("test LogLoss", round(log_loss(y_test, pred), 4))
    print("test AUC", round(roc_auc_score(y_test, pred), 4))
    print("test RIG", round(get_rig(y_train, y_test, pred, average_ctr), 4))

## LGB 학습

In [ ]:
import lightgbm as lgb

In [ ]:
model_lgb = lgb.LGBMClassifier(n_estimators=200, 
                           random_state=47, 
                           learning_rate=0.1,
                           boosting_type='gbdt', 
                           num_leaves=255, 
                           max_depth=20, 
                           min_data_in_leaf=200, 
                           zero_as_missing=True,
                           objective='binary')
model_lgb.fit(X_train, y_train)

LGBMClassifier(max_depth=20, min_data_in_leaf=200, n_estimators=200,
               num_leaves=255, objective='binary', random_state=47,
               zero_as_missing=True)

In [ ]:
pred_lgb = model_lgb.predict_proba(X_test)[:,1]
evaluate(pred_lgb)


test LogLoss 0.1269
test AUC 0.7692
test RIG 0.1035


### Finalize

In [ ]:
model_lgb_final = lgb.LGBMClassifier(n_estimators=200, 
                           random_state=47, 
                           learning_rate=0.1,
                           boosting_type='gbdt', 
                           num_leaves=255, 
                           max_depth=20, 
                           min_data_in_leaf=200, 
                           zero_as_missing=True,
                           objective='binary')

X_final = X_train.append(X_test)
y_final = y_train.append(y_test)

model_lgb_final.fit(X_final, y_final)

LGBMClassifier(max_depth=20, min_data_in_leaf=200, n_estimators=200,
               num_leaves=255, objective='binary', random_state=47,
               zero_as_missing=True)

### 모델 저장

In [ ]:
# import joblib

# # save model
# joblib.dump(model_lgb_final, 'model/lgb.pkl')

['model/lgb.pkl']

## DeepFM 학습

In [ ]:
import torch
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, X_train[feat].nunique())
                        for feat in categorical] + [DenseFeat(feat, 1, )
                                                        for feat in continuous]

# dnn feature에 일단 모든 feature 정보 넣기
dnn_feature_columns = fixlen_feature_columns

# lenear feature에도 일단 모든 feature 정보 넣기
linear_feature_columns = fixlen_feature_columns 

# feature 이름들만 따로 저장
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# train/test 데이터의 각 feature에 대응하는 컬럼을 딕셔너리로 저장
train_model_input = {name: X_train[name] for name in feature_names}
test_model_input = {name: X_test[name] for name in feature_names}

In [ ]:
device = 'cpu'
use_gpu = True
if use_gpu and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
model_dfm = DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
           dnn_activation='mish', dnn_use_bn=True, task='binary', device=device)

# optimizer, loss 설정
model_dfm.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"], )

# 모델 학습
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=2, patience=5, mode='auto')
hist = model_dfm.fit(x = train_model_input, y = y_train.values , batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

pred_dfm = model_dfm.predict(test_model_input, 512)
evaluate(pred_dfm)

cuda ready...
cuda:0
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
10s - loss:  0.1509 - binary_crossentropy:  0.1496 - auc:  0.6473 - val_binary_crossentropy:  0.1337 - val_auc:  0.6656
Epoch 2/50
9s - loss:  0.1371 - binary_crossentropy:  0.1371 - auc:  0.6661 - val_binary_crossentropy:  0.1335 - val_auc:  0.6699
Epoch 3/50
9s - loss:  0.1362 - binary_crossentropy:  0.1362 - auc:  0.6797 - val_binary_crossentropy:  0.1331 - val_auc:  0.6789
Epoch 4/50
9s - loss:  0.1356 - binary_crossentropy:  0.1356 - auc:  0.6878 - val_binary_crossentropy:  0.1325 - val_auc:  0.6891
Epoch 5/50
9s - loss:  0.1352 - binary_crossentropy:  0.1352 - auc:  0.6897 - val_binary_crossentropy:  0.1339 - val_auc:  0.6826
Epoch 6/50
9s - loss:  0.1353 - binary_crossentropy:  0.1351 - auc:  0.6925 - val_binary_crossentropy:  0.1320 - val_auc:  0.6924
Epoch 7/50
9s - loss:  0.1347 - binary_crossentropy:  0.1347 - auc:  0.6961 - val_binary_crossentropy:  0.1322 - val_auc:  0.

### Finalize

In [ ]:
device = 'cpu'
use_gpu = True
if use_gpu and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

X_final = X_train.append(X_test)
y_final = y_train.append(y_test)
final_model_input = {name: X_final[name] for name in feature_names}


# 모델 정의
model_dfm_final = DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
           dnn_activation='relu', dnn_use_bn=True, task='binary', device=device)

# optimizer, loss 설정
model_dfm_final.compile("adam", "binary_crossentropy", metrics=["binary_crossentropy", "auc"], )

# 모델 학습
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=2, patience=5, mode='auto')
hist = model_dfm_final.fit(x = final_model_input, y = y_final.values , batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

cuda ready...
cuda:0
Train on 588555 samples, validate on 147139 samples, 1150 steps per epoch
Epoch 1/50
12s - loss:  0.1411 - binary_crossentropy:  0.1411 - auc:  0.6589 - val_binary_crossentropy:  0.1354 - val_auc:  0.6814
Epoch 2/50
12s - loss:  0.1355 - binary_crossentropy:  0.1355 - auc:  0.6816 - val_binary_crossentropy:  0.1350 - val_auc:  0.6928
Epoch 3/50
12s - loss:  0.1345 - binary_crossentropy:  0.1345 - auc:  0.6926 - val_binary_crossentropy:  0.1349 - val_auc:  0.6934
Epoch 4/50
12s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6952 - val_binary_crossentropy:  0.1345 - val_auc:  0.6952
Epoch 5/50
12s - loss:  0.1338 - binary_crossentropy:  0.1337 - auc:  0.7005 - val_binary_crossentropy:  0.1347 - val_auc:  0.7022
Epoch 6/50
12s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7040 - val_binary_crossentropy:  0.1334 - val_auc:  0.7075
Epoch 7/50
12s - loss:  0.1330 - binary_crossentropy:  0.1330 - auc:  0.7083 - val_binary_crossentropy:  0.1335 - val_a

### 모델 저장

In [ ]:
# # import torch
# # model = DeepFM()
# torch.save(model_dfm_final, 'model/dfm.h5')
# # model_DFM = torch.load('경로/파일명.h5')

# CTR 예측

In [ ]:
# # load model
import torch
import joblib
model_lgb = joblib.load('model/lgb.pkl')
model_dfm = torch.load('model/dfm.h5')

### 가상 유저군 생성

In [ ]:
from itertools import product

### 위 유저들에 대해서 예측

# User Count 비교

In [ ]:
from tqdm import tqdm

In [ ]:
# 유저 정보 (median 사용)
users = {'bid_price': [50],
         'viewer_gender': [1,2],
         'viewer_age': range(10, 80),
         'viewer_chat_count': [0.0],
        'viewer_following_count': [1.0],
        'viewer_parcel_post_count': [0.0],
        'viewer_pay_count': [0.0],
        'viewer_transfer_count': [0.0]
         }

user_list = []
for i, values in enumerate(list(product(*users.values()))):
    user = dict(zip(users.keys(), values))
    user_list.append(user)

    
# 크롤링해서 얻은 게시글, 광고주 정보
candidates = ['adv_follower_count',
                  'adv_item_count',
                  'adv_review_count',
                  'content_b_pay',
                  'content_cat_1',
                  'content_comment_count',
                  'content_delivery_fee',
                  'content_likes',
                  'content_price',
                  'content_used',
                  'content_views']

user_ctr_lgb_df = pd.DataFrame()
# user_ctr_dfm_df = pd.DataFrame()

# 위에서 생성한 유저 목록에서
for demo_user in tqdm(user_list):
    candidates_df = df[candidates].copy()
    
    # 각 유저마다 특성을 입력(나이, 성별 등)
    for feat in demo_user.keys():
        candidates_df[feat] = demo_user[feat]
    
    candidates_df = candidates_df[features]

    # 전처리 진행
    candidates_df['content_cat_1'] = candidates_df['content_cat_1'].apply(lambda x: cat_label[x])
    # 학습 시 사용했던 sclaer 사용
    candidates_df[continuous] = scaler.transform(candidates_df[continuous])
    # DeepFM input 형태에 맞게
    pred_model_input = {name: candidates_df[name] for name in candidates_df.columns}

    # 각 모델별 CTR 예측
    CTR_lgb = model_lgb.predict_proba(candidates_df)[:,1]
    # CTR_dfm = model_dfm.predict(pred_model_input, 512)
    
    temp_lgb = pd.DataFrame(CTR_lgb, columns=['ctr'])
    temp_lgb['content_id'] = df['content_id'].copy()
    temp_lgb['age'] = demo_user['viewer_age']
    temp_lgb['gender'] = demo_user['viewer_gender']
    user_ctr_lgb_df = user_ctr_lgb_df.append(temp_lgb)

    # temp_dfm = pd.DataFrame(CTR_dfm, columns=['ctr'])
    # temp_dfm['content_id'] = df['content_id'].copy()
    # temp_dfm['age'] = demo_user['viewer_age']
    # temp_dfm['gender'] = demo_user['viewer_gender']
    # user_ctr_dfm_df = user_ctr_dfm_df.append(temp_dfm)

    # 모델별 평균 클릭율 출력
    print('성별:', demo_user['viewer_gender'],'나이:',  demo_user['viewer_age'])
    # print(f"클릭율 - DeepFM: {round(100*CTR_dfm.mean(),2)}%, 학습평균 : {round(100*average_ctr, 2)}%")

  1%|          | 1/140 [00:06<14:02,  6.06s/it]

성별: 1 나이: 10


  1%|▏         | 2/140 [00:12<13:57,  6.07s/it]

성별: 1 나이: 11


  2%|▏         | 3/140 [00:18<13:50,  6.06s/it]

...

성별: 2 나이: 77


 99%|█████████▉| 139/140 [15:28<00:06,  6.92s/it]

성별: 2 나이: 78


100%|██████████| 140/140 [15:35<00:00,  6.68s/it]

성별: 2 나이: 79


In [ ]:
user_ctr_lgb_df.to_csv('data/user_lgb.csv', index=False)

In [ ]:
# 기준 모델, 유저정보 입력
model_name = 'dfm'
user_age = 20
user_gender = '남성'
ctr_name = f'{model_name}_{user_age}세_{user_gender}'
# 기준에 따라 정렬
df = df.sort_values(ctr_name, ascending=False)

# HTML로 시각화
html = ""
for idx, row in df[df.is_ad == True].iloc[0:20].iterrows():
    html += f'''
        <div style="display:inline-block;min-width:150px;max-width:150px;vertical-align:top">
        <ul>
            <li>광고명: {row.content_name}</li>
            <li>가격: {row.content_price}</li>
            <li>카테고리: {row.content_cat_1}</li>
            <li>CTR: {round(100*row[ctr_name], 2)}%</li>
        </ul>
        <img src="https://media.bunjang.co.kr/product/{row.content_id}_..." style="width:150px;">
        </div>
        '''
        
display(HTML(html))